In [1]:
import pandas as pd
import re
import base64
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
from collections import Counter
import nltk
from transformers import BertTokenizer
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from email.header import decode_header
from quopri import decodestring
from html.parser import HTMLParser


/home/rezett/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download NLTK stopwords
nltk.download('stopwords')

# Initialize stop words and stemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

[nltk_data] Downloading package stopwords to /home/rezett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/rezett/miniconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import re
from quopri import decodestring
from email.header import decode_header

def clean_text(text):
    if text is None:
        return None
    
    # Step 1: Remove email headers like "Content-Type", "Content-Transfer-Encoding", boundaries, etc.
    # This regex removes lines that start with boundary markers or headers.
    text = re.sub(r'(--[-_a-zA-Z0-9]+|Content-[^:]+:[^\n]+\n)', '', text)
    
    # Step 2: Decode quoted-printable encoding artifacts
    try:
        # First, replace soft line breaks that look like =\n and remove remaining = signs.
        text = text.replace('=\n', '')  # Soft line breaks in quoted-printable
        
        # Decode quoted-printable encoding using `decodestring` from quopri
        text = decodestring(text).decode('utf-8', errors='ignore')  # Decode quoted-printable to UTF-8
    except Exception as e:
        print(f"Error decoding quoted-printable text: {e}")
        pass  # Handle the case where decoding fails
    
    # Step 3: Remove HTML tags if any remain
    text = re.sub(r'<[^>]+>', '', text)
    
    # Step 4: Normalize whitespaces and remove extra spaces or newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Step 5: Decode any additional email headers if present (like encoded subjects)
    text = decode_headers(text)

    # Step 6: Strip leading and trailing whitespaces
    text = text.strip()
    
    return text

# Optional helper function to decode headers in case they are encoded
def decode_headers(header_text):
    if header_text:
        # Decode headers like "=?utf-8?Q?...?="
        decoded_parts = decode_header(header_text)
        decoded_string = ''
        for part, encoding in decoded_parts:
            if isinstance(part, bytes):
                if encoding is None:
                    decoded_string += part.decode('utf-8', errors='ignore')
                else:
                    decoded_string += part.decode(encoding, errors='ignore')
            else:
                decoded_string += part
        return decoded_string
    return header_text


In [4]:
def clean_text(text):
    if text is None:
        return None

    # Step 1: Remove quoted-printable encoding artifacts
    text = text.replace('=\n', '')  # Remove soft line breaks
    text = text.replace('=', '')  # Remove remaining '=' characters

    # Step 2: Remove MIME boundaries, content-type, and charset declarations
    mime_pattern = r'(--[-_a-zA-Z0-9]+|Content-[^:]+:[^\n]+\n|charset="[^"]+"|MIME-Version:[^\n]+\n)'
    text = re.sub(mime_pattern, '', text)

    # Step 3: Remove base64-like encoded binary content
    base64_pattern = r'[A-Za-z0-9+/]{40,}={0,2}'
    text = re.sub(base64_pattern, '', text)

    # Step 4: Use BeautifulSoup to remove HTML tags and decode entities
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Step 5: Remove corrupted MIME artifacts or non-readable sequences
    text = re.sub(r'\.[A-Za-z0-9]{7,}', '', text)

    # Step 6: Replace multiple spaces, newlines, or tabs with a single space
    text = re.sub(r'\s+', ' ', text)

    # Step 7: Strip leading and trailing whitespace
    text = text.strip()

    return text

In [ ]:
df_phishing = pd.read_csv('../jose_phishing.csv')
df_ham = pd.read_csv('../trec_train.csv')

# Apply the cleaning function to the extracted text
df_phishing['message'] = df_phishing['body'].apply(clean_text)
df_ham['message'] = df_ham['body'].apply(clean_text)

# Check the number of original email bodies
original_count = df_phishing.shape[0]

# Remove duplicates based on the cleaned_body column
df_cleaned = df_phishing.drop_duplicates(subset='message')

# Check the number of email bodies after removing duplicates
after_removal_count = df_cleaned.shape[0]

# Output the results
print(f"Original number of email bodies: {original_count}")
print(f"Number of email bodies after removing duplicates: {after_removal_count}")

# Select only the clean_text and label columns
df_phishing = df_cleaned[['message', 'label']]
df_ham = df_ham[['message', 'label']]

# Save to a new CSV file
df_phishing.to_csv('cleaned_phishing_emails.csv', index=False, errors='ignore')
df_ham.to_csv('cleaned_ham_emails.csv', index=False)


/tmp/ipykernel_11558/654631364.py:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
/tmp/ipykernel_11558/654631364.py:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


Original number of email bodies: 2544
Number of email bodies after removing duplicates: 2164
0    You will not see this in a MIME-aware mail rea...
1    This is a multi-part message in MIME format. c...
2    You will not see this in a MIME-aware mail rea...
3    Mail Quota: (98% Full) Attention: jose Your em...
4    You will not see this in a MIME-aware mail rea...
Name: message, dtype: object
0    plot(fn,-1,1) could be helpful, hth, Ingmar On...
1    CNN Alerts: bush Alert Name: bush Bush talks t...
2    Hello sm1lies, This week we've got $34,050 in ...
3    Hi, this evening I tried to add Festival and F...
4    Napster Stock Quote Notification Stock informa...
Name: message, dtype: object
0    Important Security Message You will not see th...
1    NEW PDF MESSAGE FROM AMERICAN EXPRESS ONLINE F...
2    Confirm Your recent Transactions You will not ...
3    =?UTF-8?B?WW91ciBFbWFpbCDinIkg?=jose@monkey.or...
4    confirm This Transaction You will not see this...
Name: message, dtype: ob

/tmp/ipykernel_11558/4085025145.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['message'] = df_cleaned['subject'].fillna('') + ' ' + df_cleaned['message']
